In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
nameCSV = 'innis_qc.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]

In [3]:
netCDF4.num2date(tt[0:200],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 3, 31, 8, 13, 38, 15623),
       datetime.datetime(1985, 3, 31, 8, 13, 38, 992192),
       datetime.datetime(1985, 3, 31, 8, 13, 39, 976566),
       datetime.datetime(1985, 3, 31, 8, 13, 40, 953124),
       datetime.datetime(1985, 3, 31, 8, 13, 41, 929683),
       datetime.datetime(1985, 3, 31, 8, 13, 42, 914067),
       datetime.datetime(1985, 3, 31, 8, 13, 43, 890626),
       datetime.datetime(1985, 3, 31, 8, 13, 44, 875000),
       datetime.datetime(1985, 3, 31, 8, 13, 45, 851559),
       datetime.datetime(1985, 3, 31, 8, 13, 46, 835933),
       datetime.datetime(1985, 3, 31, 8, 13, 47, 812502),
       datetime.datetime(1985, 3, 31, 8, 13, 48, 789061),
       datetime.datetime(1985, 3, 31, 8, 13, 49, 773435),
       datetime.datetime(1985, 3, 31, 8, 13, 50, 750003),
       datetime.datetime(1985, 3, 31, 8, 13, 51, 734378),
       datetime.datetime(1985, 3, 31, 8, 13, 52, 710936),
       datetime.datetime(1985, 3, 31, 8, 13, 53, 695310),
       datetime

In [17]:
#TEST TO SEE ALL HIGH VALUES
idd = np.where(np.logical_and(wh>4, wh<5))[0]

for k in range(len(idd)):
    print wh[idd[k]], netCDF4.num2date(tt[idd[k]],u'days since 1985-01-01 00:00:00 UTC')

4.0600004 1988-06-25 20:13:07.953122
4.348 1988-06-25 20:13:08.937496
4.743 1988-06-25 20:13:09.921870
4.829 1988-06-25 20:13:10.890624
4.8770003 1988-06-25 20:13:12.859372
4.123 1995-08-11 00:30:57.518802
4.049000299999999 1995-08-17 00:25:09.008059
4.049000299999999 1995-08-17 00:25:12.229979
4.8900003 1997-03-09 00:27:52.385990
4.934 1997-03-09 00:27:55.327153
4.049000299999999 1998-01-17 00:58:07.873045
4.049000299999999 1998-01-17 00:58:08.946777
4.049000299999999 1998-01-17 00:58:11.095216
4.049000299999999 1998-01-17 00:58:13.243163
4.049000299999999 1998-01-17 00:58:14.316894
4.7710004 1998-01-18 00:27:49.579105
4.998 1998-01-18 00:27:52.520268
4.901 1998-01-18 00:27:53.500488
4.5340004 1998-01-18 00:27:54.480960
4.7390003 1998-01-18 00:27:55.461180
4.188 1998-01-18 00:27:56.441893
4.3500004 1998-01-18 00:27:57.422123
4.263 1998-01-18 00:28:03.304690
4.049000299999999 1999-05-06 11:03:10.195311
4.0470004 2000-02-27 00:27:52.468016
4.0950003 2009-01-13 08:16:55.679688
4.184 2013

In [5]:
tt1 = netCDF4.num2date(tt,u'days since 1985-01-01 00:00:00 UTC')
print tt1[0], wh[0]
print netCDF4.num2date(max(tt),u'days since 1985-01-01 00:00:00 UTC')

1985-03-31 08:13:38.015623 1.9050001
2019-02-20 11:54:42.070312


In [6]:
#MAKE MEAN SATELLITE TRACK. DOESNT SEEM TO WORK??
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [7]:
print netCDF4.num2date(tt[days[0]],u'days since 1985-01-01 00:00:00 UTC')#same as first record

# meanwh is the mean wh of each day (satellite track)
meanwh = []

for k in range(len(days)-1):
    meanwh.append(np.mean(wh[days[k]:days[k+1]])) 
    
print np.mean(meanwh), 'mean from each satellite track'
print np.mean(wh), 'mean from each record'
print len(wh), 'number of records'
print len(meanwh), 'number of satellite tracks'

#slightly different mean wh depending on if done by day, or individual records (each second)

1985-03-31 08:13:38.015623
1.4000161397591937 mean from each satellite track
1.4150025589470594 mean from each record
46335 number of records
2516 number of satellite tracks


In [8]:
netCDF4.num2date(days,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 1, 0, 0), real_datetime(1985, 1, 19, 0, 0),
       real_datetime(1985, 1, 23, 0, 0), ...,
       real_datetime(2111, 8, 28, 0, 0), real_datetime(2111, 9, 14, 0, 0),
       real_datetime(2111, 10, 4, 0, 0)], dtype=object)

In [9]:
len(days)

2517

MEAN MONTHLY VALUES

In [10]:
for k in range (len(days)):
    print netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')

1985-01-01 00:00:00
1985-01-19 00:00:00
1985-01-23 00:00:00
1985-01-24 00:00:00
1985-01-29 00:00:00
1985-02-03 00:00:00
1985-02-09 00:00:00
1985-02-18 00:00:00
1985-03-01 00:00:00
1985-03-11 00:00:00
1985-03-12 00:00:00
1985-03-16 00:00:00
1985-03-27 00:00:00
1985-04-14 00:00:00
1985-04-23 00:00:00
1985-05-03 00:00:00
1985-05-08 00:00:00
1985-05-22 00:00:00
1985-05-31 00:00:00
1985-06-16 00:00:00
1985-07-05 00:00:00
1985-07-12 00:00:00
1985-07-25 00:00:00
1985-07-29 00:00:00
1985-08-15 00:00:00
1985-08-31 00:00:00
1985-09-18 00:00:00
1985-09-30 00:00:00
1985-10-02 00:00:00
1985-10-04 00:00:00
1985-10-06 00:00:00
1985-10-12 00:00:00
1985-10-15 00:00:00
1985-10-22 00:00:00
1985-11-01 00:00:00
1985-11-03 00:00:00
1985-11-09 00:00:00
1985-11-11 00:00:00
1985-11-18 00:00:00
1985-11-24 00:00:00
1985-11-26 00:00:00
1985-12-03 00:00:00
1985-12-04 00:00:00
1985-12-06 00:00:00
1985-12-12 00:00:00
1985-12-19 00:00:00
1985-12-26 00:00:00
1985-12-31 00:00:00
1986-01-07 00:00:00
1986-01-14 00:00:00


2033-05-05 00:00:00
2033-05-11 00:00:00
2033-05-26 00:00:00
2033-06-10 00:00:00
2033-06-16 00:00:00
2033-06-29 00:00:00
2033-07-11 00:00:00
2033-07-21 00:00:00
2033-08-04 00:00:00
2033-08-18 00:00:00
2033-08-28 00:00:00
2033-09-03 00:00:00
2033-09-18 00:00:00
2033-10-03 00:00:00
2033-10-08 00:00:00
2033-10-17 00:00:00
2033-10-28 00:00:00
2033-11-03 00:00:00
2033-11-14 00:00:00
2033-11-20 00:00:00
2033-11-26 00:00:00
2033-12-06 00:00:00
2033-12-21 00:00:00
2033-12-28 00:00:00
2034-01-02 00:00:00
2034-01-09 00:00:00
2034-01-15 00:00:00
2034-01-30 00:00:00
2034-02-14 00:00:00
2034-03-01 00:00:00
2034-03-12 00:00:00
2034-03-27 00:00:00
2034-04-05 00:00:00
2034-04-20 00:00:00
2034-05-01 00:00:00
2034-05-07 00:00:00
2034-05-22 00:00:00
2034-06-06 00:00:00
2034-06-12 00:00:00
2034-06-27 00:00:00
2034-07-07 00:00:00
2034-07-13 00:00:00
2034-07-29 00:00:00
2034-08-17 00:00:00
2034-09-01 00:00:00
2034-09-13 00:00:00
2034-09-19 00:00:00
2034-10-04 00:00:00
2034-10-08 00:00:00
2034-10-23 00:00:00


2050-09-18 00:00:00
2050-10-17 00:00:00
2050-11-13 00:00:00
2050-12-01 00:00:00
2050-12-21 00:00:00
2051-01-04 00:00:00
2051-02-08 00:00:00
2051-02-26 00:00:00
2051-03-15 00:00:00
2051-04-02 00:00:00
2051-04-17 00:00:00
2051-04-27 00:00:00
2051-05-17 00:00:00
2051-06-06 00:00:00
2051-06-23 00:00:00
2051-07-13 00:00:00
2051-07-28 00:00:00
2051-08-12 00:00:00
2051-09-01 00:00:00
2051-09-21 00:00:00
2051-10-11 00:00:00
2051-10-26 00:00:00
2051-11-24 00:00:00
2051-12-14 00:00:00
2052-01-03 00:00:00
2052-01-20 00:00:00
2052-02-03 00:00:00
2052-02-23 00:00:00
2052-03-09 00:00:00
2052-03-29 00:00:00
2052-04-16 00:00:00
2052-05-06 00:00:00
2052-05-21 00:00:00
2052-06-09 00:00:00
2052-06-25 00:00:00
2052-07-15 00:00:00
2052-08-04 00:00:00
2052-08-21 00:00:00
2052-09-10 00:00:00
2052-09-30 00:00:00
2052-10-15 00:00:00
2052-10-30 00:00:00
2052-11-19 00:00:00
2052-12-10 00:00:00
2052-12-27 00:00:00
2053-01-15 00:00:00
2053-02-19 00:00:00
2053-03-02 00:00:00
2053-03-13 00:00:00
2053-03-23 00:00:00


2087-08-09 00:00:00
2087-09-08 00:00:00
2087-09-23 00:00:00
2087-11-03 00:00:00
2087-11-20 00:00:00
2087-12-24 00:00:00
2088-01-28 00:00:00
2088-02-11 00:00:00
2088-03-06 00:00:00
2088-03-24 00:00:00
2088-04-21 00:00:00
2088-04-24 00:00:00
2088-06-08 00:00:00
2088-07-19 00:00:00
2088-08-28 00:00:00
2088-09-13 00:00:00
2088-09-29 00:00:00
2088-11-25 00:00:00
2088-12-23 00:00:00
2089-01-09 00:00:00
2089-01-25 00:00:00
2089-01-27 00:00:00
2089-02-10 00:00:00
2089-03-20 00:00:00
2089-04-29 00:00:00
2089-05-24 00:00:00
2089-06-06 00:00:00
2089-07-16 00:00:00
2089-08-21 00:00:00
2089-09-03 00:00:00
2089-09-21 00:00:00
2089-10-30 00:00:00
2089-12-14 00:00:00
2089-12-30 00:00:00
2089-12-31 00:00:00
2090-01-17 00:00:00
2090-02-26 00:00:00
2090-04-05 00:00:00
2090-05-02 00:00:00
2090-05-07 00:00:00
2090-06-16 00:00:00
2090-07-24 00:00:00
2090-08-10 00:00:00
2090-09-17 00:00:00
2090-10-04 00:00:00
2090-11-13 00:00:00
2090-11-16 00:00:00
2090-12-11 00:00:00
2091-01-16 00:00:00
2091-03-14 00:00:00


In [11]:
netCDF4.num2date(meanwh,u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 1, 3, 3, 53, 7, 207200),
       datetime.datetime(1985, 1, 1, 6, 59, 2, 401080),
       datetime.datetime(1985, 1, 3, 3, 56, 9, 600000), ...,
       datetime.datetime(1985, 1, 2, 5, 21, 37, 443240),
       datetime.datetime(1985, 1, 2, 10, 59, 31, 204066),
       datetime.datetime(1985, 1, 2, 4, 5, 5, 283456)], dtype=object)

In [12]:
janWh = []
febWh = []
marWh = []
aprWh = []
mayWh = []
junWh = []
julWh = []
augWh = []
sepWh = []
octWh = []
novWh = []
decWh = []

for k in range(len(days)):
    dt = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')
    if dt.month == 1:
        janWh.append(k)
    if dt.month == 2:
        febWh.append(k)
    if dt.month == 3:
        marWh.append(k)
    if dt.month == 4:
        aprWh.append(k)
    if dt.month == 5:
        mayWh.append(k)
    if dt.month == 6:
        junWh.append(k)
    if dt.month == 7:
        julWh.append(k)
    if dt.month == 8:
        augWh.append(k)
    if dt.month == 9:
        sepWh.append(k)
    if dt.month == 10:
        octWh.append(k)
    if dt.month == 11:
        novWh.append(k)
    if dt.month == 12:
        decWh.append(k)

In [13]:
febWh

[5,
 6,
 7,
 52,
 53,
 54,
 55,
 75,
 76,
 96,
 115,
 136,
 137,
 157,
 176,
 177,
 196,
 216,
 236,
 237,
 255,
 273,
 293,
 294,
 314,
 333,
 334,
 352,
 353,
 370,
 371,
 389,
 407,
 408,
 424,
 425,
 445,
 463,
 464,
 485,
 503,
 520,
 521,
 538,
 539,
 557,
 576,
 593,
 594,
 612,
 613,
 631,
 632,
 649,
 650,
 670,
 671,
 688,
 706,
 707,
 725,
 726,
 727,
 747,
 767,
 786,
 787,
 806,
 807,
 826,
 827,
 846,
 866,
 887,
 888,
 905,
 922,
 938,
 939,
 940,
 958,
 959,
 993,
 1022,
 1060,
 1061,
 1062,
 1097,
 1098,
 1127,
 1128,
 1158,
 1159,
 1191,
 1192,
 1193,
 1194,
 1195,
 1221,
 1222,
 1241,
 1258,
 1279,
 1280,
 1302,
 1303,
 1320,
 1321,
 1341,
 1342,
 1362,
 1363,
 1383,
 1384,
 1403,
 1422,
 1423,
 1442,
 1443,
 1462,
 1482,
 1483,
 1502,
 1503,
 1537,
 1570,
 1587,
 1588,
 1607,
 1608,
 1629,
 1630,
 1648,
 1666,
 1686,
 1687,
 1705,
 1726,
 1727,
 1746,
 1763,
 1764,
 1783,
 1799,
 1836,
 1837,
 1851,
 1870,
 1887,
 1888,
 1905,
 1922,
 1923,
 1924,
 1942,
 1943,
 196

In [14]:
janW = wh[janWh]
febW = wh[febWh]
marW = wh[marWh]
aprW = wh[aprWh]
mayW = wh[mayWh]
junW = wh[junWh]
julW = wh[julWh]
augW = wh[augWh]
sepW = wh[sepWh]
octW = wh[octWh]
novW = wh[novWh]
decW = wh[decWh]


print 'Month    Min             Max             Mean'
print 'Jan', janW.min(),janW.max(),janW.mean()
print 'Feb', febW.min(),febW.max(),febW.mean()
print 'Mar', marW.min(),marW.max(),marW.mean()
print 'Apr', aprW.min(),aprW.max(),aprW.mean()
print 'May', mayW.min(),mayW.max(),mayW.mean()
print 'Jun', junW.min(),junW.max(),junW.mean()
print 'Jul', julW.min(),julW.max(),julW.mean()
print 'Aug', augW.min(),augW.max(),augW.mean()
print 'Sep', sepW.min(),sepW.max(),sepW.mean()
print 'Oct', octW.min(),octW.max(),octW.mean()
print 'Nov', novW.min(),novW.max(),novW.mean()
print 'Dec', decW.min(),decW.max(),decW.mean()

Month    Min             Max             Mean
Jan 0.358 3.627 1.5331147411467891
Feb 0.38500002 3.627 1.5497549557352943
Mar 0.358 3.522 1.5208597863800908
Apr 0.358 3.7330002999999996 1.5788454234536085
May 0.32700002 3.522 1.591418243590909
Jun 0.27899999999999997 3.2060000000000004 1.4940096774519231
Jul 0.25 4.0600004 1.5456186651162793
Aug 0.25 4.743 1.5414020180882355
Sep 0.47100002 5.05 1.6637560539712917
Oct 0.317 4.8770003 1.526428628965517
Nov 0.35600000000000004 3.627 1.491387314019608
Dec 0.358 3.7330002999999996 1.5260922262211984


yearly trend

In [37]:
dt.year(1117)

TypeError: 'int' object is not callable

In [29]:
y1985 = []
y1986 = []
y1987 = []


for k in range(len(days)):
    dt = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')
    if dt.year == 1985:
        y1985.append(k)
    if dt.year == 1986:
        y1986.append(k)
    if dt.year == 1987:
        y1987.append(k)


In [32]:
netCDF4.num2date(y1985,u'days since 1985-01-01 00:00:00 UTC')

IndexError: list index out of range

In [31]:
y1985 = []
y1986 = []
y1987 = []

wh1985 = wh[y1985]
wh1986 = wh[y1986]
wh1987 = wh[y1987]

print 'Year    Min             Max             Mean'
print '1985', wh1985.min(),wh1985.max(),wh1985.mean()
print '1986', wh1986.min(),wh1986.max(),wh1986.mean()
print '1987', wh1987.min(),wh1987.max(),wh1987.mean()

Year    Min             Max             Mean
1985

ValueError: zero-size array to reduction operation minimum which has no identity

In [23]:
# months = []
# for k in range (len(days)):
#     t_month = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')
#     if k ==0:
#         months.append(0)
#         mm = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')
#         it = 0
#     else:
#         if t_month.month != mm.month: #if day 1 is not equal to day 2, then append
#             #print dd.day,t1.day
#             days.append(k)
#             it += 1
#             mm = netCDF4.num2date(days[k],u'days since 1985-01-01 00:00:00 UTC')










#MAKE MEAN SATELLITE TRACK. DOESNT SEEM TO WORK??
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.month != dd.month: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [24]:
months

[0]

In [22]:
for k in range(len(days)):
    print netCDF4.num2date(mon[k],u'days since 1985-01-01 00:00:00 UTC')
    
    
    
    
    
    
#TEST TO SEE ALL HIGH VALUES
idd = np.where(np.logical_and(wh>4, wh<5))[0]

for k in range(len(idd)):
    print wh[idd[k]], netCDF4.num2date(tt[idd[k]],u'days since 1985-01-01 00:00:00 UTC')

TypeError: can't compare real_datetime to int

In [ ]:
netCDF4.num2date(days,u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
#why does this give me different mean wh value?

wh_ = wh[days]
qc_ = qc[days]
lon_ = lon[days]
tt_ = tt[days]
lat_ = lat[days]

print max(wh_)
print min(wh_)
print len(tt)
print len(days)
print np.mean(wh_)

In [ ]:
#for k in range(len(days)):
#    if wh_[k]>5:
#        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
#This should be average per track, wh above 7.

for k in range(len(wh_)):
    if wh_[k]>7:
        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
daywh = [] #daystart
for k in range(len(tt)):
    if wh[k]<10:
        t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        if k == 0: #get day 0. Then else: every day after that
            daywh.append(0)
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            it = 0
        else:
            if t1.day != dd.day: #if day 1 is not equal to day 2, then append
                #print dd.day,t1.day
                daywh.append(k)
                it += 1
                dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
len(daywh)

In [ ]:
wh2 = wh[daywh]
qc2 = qc[daywh]
lon2 = lon[daywh]
tt2 = tt[daywh]
lat2 = lat[daywh]

print max(wh2)
print min(wh2)
print len(tt2)
print np.mean(wh2)

In [ ]:
for k in range(len(wh2)):
    if wh2[k]>7:
        print wh2[k], netCDF4.num2date(tt2[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
netCDF4.num2date(12087.8427231626124,u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
print np.mean(wh2), 'mean'
print max(wh2), 'max'
print min(wh2), 'min'

# for p in range(len(timing)):
            #print(u,' - ',k,'/',len(urlON),' - ',p,'/',len(timing))
            if timing[p] >= start_date and timing[p] <= end_date:
                if lats[p]>latmin and lats[p]<latmax and lons[p]>lonmin and lons[p]<lonmax:
                    if np.where(np.logical_and(0<wh[p]<10,qc[p]==1)):
                        boxLat.append(lats[p])